# 🛠️ Notebook 02: Preprocessing & Feature Engineering Avanzato

**Obiettivo:** Trasformare i dati grezzi esplorati nell'EDA in un dataset pronto per l'addestramento dei modelli di Machine Learning, risolvendo le criticità emerse (skewness, sbilanciamento, non-linearità).

### 🚀 Roadmap delle Attività
Basandoci sulle evidenze del *Notebook 01*, implementeremo le seguenti strategie:

1.  **Target Engineering (6 → 3 Classi):** Accorperemo le classi per risolvere lo sbilanciamento estremo (<1% Hazardous) e migliorare la separabilità.
2.  **Feature Selection:** Esclusione rigorosa di `AQI Value` per evitare *Data Leakage*.
3.  **Log-Trasformazione:** Applicazione di `log1p` su CO, NO2 e PM2.5 per correggere la skewness elevata (> 3.0) e aiutare i modelli lineari.
4.  **Domain-Driven Feature Engineering:** Creazione di nuove variabili basate sulla chimica atmosferica (es. *Traffic Index*, *Smog Index*) per catturare interazioni non lineari.
5.  **Stratified Split:** Suddivisione Train/Val/Test che rispetti rigorosamente le proporzioni delle classi di rischio.


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import os

# Configurazione ambiente
output_dir = '../output'
data_dir = '../data'
os.makedirs(output_dir, exist_ok=True)

# Caricamento Dataset
df = pd.read_csv(f'{data_dir}/global_air_pollution_dataset.csv')
print(f"Dataset caricato: {df.shape[0]} righe, {df.shape[1]} colonne")


Dataset caricato: 23463 righe, 12 colonne


## 1. Definizione del nuovo target (3 classi)

Dopo l'EDA abbiamo visto che le 6 classi originali (`AQI Category`) sono fortemente sbilanciate, in particolare **Very Unhealthy** e **Hazardous** hanno meno del 2% dei campioni.

Per ottenere metriche più affidabili, aggreghiamo le classi in 3 livelli di rischio:

- **Safe** → Good
- **Acceptable** → Moderate
- **Hazardous** → Unhealthy for Sensitive Groups, Unhealthy, Very Unhealthy, Hazardous

Questa scelta mantiene il significato clinico delle classi e rende il problema più adatto al training ML.


In [7]:
# Mappatura semantica
mapping_3_classes = {
    'Good': 'Safe',
    'Moderate': 'Acceptable',
    'Unhealthy for Sensitive Groups': 'Hazardous',
    'Unhealthy': 'Hazardous',
    'Very Unhealthy': 'Hazardous',
    'Hazardous': 'Hazardous'
}

# Applicazione mapping
df['AQI_Class_Label'] = df['AQI Category'].map(mapping_3_classes)

# Encoding numerico per i modelli (0, 1, 2)
class_map = {'Safe': 0, 'Acceptable': 1, 'Hazardous': 2}
df['AQI_Class_Encoded'] = df['AQI_Class_Label'].map(class_map)

# Verifica distribuzione
print("DISTRIBUZIONE NUOVO TARGET:")
print(df['AQI_Class_Label'].value_counts(normalize=True).mul(100).round(2))


DISTRIBUZIONE NUOVO TARGET:
AQI_Class_Label
Safe          42.35
Acceptable    39.34
Hazardous     18.31
Name: proportion, dtype: float64


## 2. Feature Engineering & Trasformazioni

In questa fase arricchiamo il dataset per aiutare i modelli a catturare pattern complessi.

### A. Gestione della Skewness (Log-Trasformazione)
Il *Notebook 01* ha mostrato che **CO** (Skew=23.0) e **PM2.5** hanno distribuzioni a coda lunga. I modelli lineari (Logistic Regression) soffrono questi outlier.
*   **Azione:** Applichiamo `np.log1p(x)` (logaritmo naturale di x+1) per "comprimere" i picchi e rendere la distribuzione più gaussiana.

### B. Nuove Feature "Chimiche"
Le correlazioni lineari tra inquinanti sono basse (<0.5). Creiamo feature combinate per catturare fenomeni specifici:
1.  **Traffic Index (`CO * NO2`):** Entrambi sono marker tipici della combustione veicolare. Un alto valore congiunto indica traffico intenso.
2.  **Smog Index (`Ozone * NO2`):** L'ozono troposferico si forma spesso dalla reazione degli ossidi di azoto col sole.
3.  **Max Pollutant:** Poiché l'AQI è definito dal valore peggiore tra gli inquinanti, estraiamo esplicitamente questo valore massimo.
4.  **High PM2.5 Flag:** Una soglia binaria (PM2.5 > 100) per segnalare esplicitamente la zona di pericolo, aiutando i modelli a trovare il "gradino" critico.


In [8]:
# 1. Selezione Feature Base (NO AQI Value per evitare Leakage)
base_features = ['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value']
X = df[base_features].copy()
y = df['AQI_Class_Encoded']

print(f"Features base: {base_features}")

# 2. Log-Trasformazione (Skewness reduction)
for col in base_features:
    X[f'log_{col}'] = np.log1p(X[col])

# 3. Feature Engineering Avanzato
# Interazioni chimiche
X['Traffic_Index'] = X['CO AQI Value'] * X['NO2 AQI Value']
X['Smog_Index'] = X['Ozone AQI Value'] * X['NO2 AQI Value']

# Logiche di dominio (AQI calculation logic)
X['Max_Pollutant'] = X[base_features].max(axis=1)
X['Pollutant_Range'] = X[base_features].max(axis=1) - X[base_features].min(axis=1)

# Soglie critiche (Domain Knowledge)
X['High_PM25_Flag'] = (X['PM2.5 AQI Value'] > 100).astype(int)

print(f"✅ Totale Features create: {X.shape[1]}")
print(f"Lista: {list(X.columns)}")

Features base: ['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value']
✅ Totale Features create: 13
Lista: ['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value', 'log_CO AQI Value', 'log_Ozone AQI Value', 'log_NO2 AQI Value', 'log_PM2.5 AQI Value', 'Traffic_Index', 'Smog_Index', 'Max_Pollutant', 'Pollutant_Range', 'High_PM25_Flag']


## 3. Suddivisione Stratificata (Train / Val / Test)

Dato che la classe *Hazardous* rappresenta circa il 18% del dataset, un *random split* semplice potrebbe creare set di Test dove questa classe è sottorappresentata, falsando la valutazione.

**Soluzione:** Usiamo `stratify=y`.
Questo garantisce che la proporzione (Safe 42% / Acceptable 40% / Hazardous 18%) sia identica in tutti e tre i set:
*   **Training Set (70%):** Per addestrare i modelli.
*   **Validation Set (15%):** Per il tuning degli iperparametri.
*   **Test Set (15%):** Per la valutazione finale (mai visto dai modelli).


In [9]:
# Split 1: Train (70%) vs Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

# Split 2: Validation (15%) vs Test (15%) - (metà del 30% restante)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

print(f"Dimensione Train: {X_train.shape} (70%)")
print(f"Dimensione Val:   {X_val.shape}   (15%)")
print(f"Dimensione Test:  {X_test.shape}  (15%)")


Dimensione Train: (16424, 13) (70%)
Dimensione Val:   (3519, 13)   (15%)
Dimensione Test:  (3520, 13)  (15%)


## 4. Standardizzazione (Scaling)

I modelli come SVM e Regressione Logistica (e in parte anche le Reti Neurali) sono sensibili alla scala delle feature.
*   *Esempio:* `Traffic_Index` può arrivare a 10.000, mentre `log_CO` arriva a 5. Senza scaling, il modello darebbe peso solo al Traffic Index.

**Procedura Corretta:**
Usiamo `StandardScaler` (Z-Score normalization).
⚠️ **Importante:** Il fit (`calcolo media/std`) viene fatto **SOLO sul Training Set** per evitare *Data Leakage*. Validation e Test vengono trasformati usando le statistiche del Train.


In [10]:
scaler = StandardScaler()

# Fit solo su Train
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)

# Transform su Val e Test
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

# Reset degli indici per allineamento
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Salvataggio finale
data_packet = {
    'X_train': X_train_scaled, 'y_train': y_train,
    'X_val': X_val_scaled, 'y_val': y_val,
    'X_test': X_test_scaled, 'y_test': y_test,
    'class_map': class_map,
    'feature_names': list(X.columns)
}

outfile = f'{data_dir}/processed_data.pkl'
with open(outfile, 'wb') as f:
    pickle.dump(data_packet, f)

print(f"✅ Dataset processato salvato in: {outfile}")
print("Pronto per il Notebook 03 (Modellazione)")


✅ Dataset processato salvato in: ../data/processed_data.pkl
Pronto per il Notebook 03 (Modellazione)
